In [ ]:
api_key = "ADD API KEY HERE"

# 1.Install Lib

In [ ]:
!pip install langchain==0.3.7
!pip install langchain-openai==0.2.9
!pip install langchain_community==0.3.13

# 2.Create Simple RAG

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

## 2.1. Indexing

**Step 1**: Load

In [ ]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://huyenchip.com/2021/02/27/why-not-join-a-startup.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
print(docs[0].page_content[100:500])

   • Chip Huyen


In 2018, I wrote Career advice for recent Computer Science graduates about joining a big company instead of a startup after college.
In 2019, when I left NVIDIA, I wrote Lessons learned after my first full-time job about leaving a big company for a startup.
Now that I’ve left my first full-time job at a startup, I want to revisit the topic. This is based on some personal experien


**Step 2: Split**  

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=10)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

64

In [ ]:
all_splits[1].page_content

'In 2018, I wrote Career advice for recent Computer Science graduates about joining a big company instead of a startup after college.\nIn 2019, when I left NVIDIA, I wrote Lessons learned after my first full-time job about leaving a big company for a startup.'

In [ ]:
all_splits[8].page_content

'Nobody told me how to spend my time, but when everyone else worked over the weekend and responded to my Slack messages any time of the night, I wanted to do the same. Nobody forced me to take on a hefty task that would require me to cancel plans with friends, but I also knew that everybody else had their hands full and if I didn’t do it, we wouldn’t be able to finish this feature on time and the'

In [ ]:
all_splits[9].page_content

'and the company would lose a contract or even die.'

**Step 3:** Storage

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Setup Vector Database
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key)
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
_ = vector_store.add_documents(documents=all_splits)

## 2.2 Retrieval

In [ ]:
question = "What is the main reason to join a startup company?"

**Step 1** Retrival

In [ ]:
retrieved_docs = vector_store.similarity_search(question, k=2)
retrieved_docs_str = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
print(retrieved_docs_str)

One reason to join a startup
There are many reasons not to join a startup. You just need one good reason to join. The reason is that it’s all worth it. I once read somewhere that joining startups is the only way to get 15 years of experience in 5 years. I couldn’t agree more.

This can actually be a reason why you should join a startup – you’ll learn to stick out for yourself. If you know what you want and are proactive about it, you might even be able to assume a leadership role after a very short period of time. Joining a startup can also be a much faster way to climb big co’s ladders. For example, after 3 years at a startup, you became a team lead of 15 engineers,


**Step 2**: Generate answer

In [ ]:
template = """
Answer my question based on the context.

Question: {question}
Context: {context}
"""
prompt = PromptTemplate(input_variables=["question", "context"], template=template)

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
messages = prompt.invoke({"question": question, "context": retrieved_docs})
response = llm.invoke(messages)

In [ ]:
print(response.content)

The main reason to join a startup company is that it offers the opportunity to gain extensive experience in a short amount of time, potentially achieving what would take many years in a traditional job setting. Joining a startup can accelerate personal and professional growth, allowing individuals to take on leadership roles and responsibilities sooner than they might in larger companies.
